# 7 WorkFlow

### 7.1 Objetivo

Presentar un workflow/pipeline completo al que los estudiantes deberán enriquecer

#### 7.2  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab
*   Bajar el **dataset_historico** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab



*  Si usted eligió modalidad *Gerencial* entonces NO debe modificar nada, su liderazgo logró que otras trabajaron por usted
*  Si usted eligió modalidad de *Analista Junior*, entonces donde dice archivo="gerencial_competencia_2025.csv.gz"  lo debe cambiar por  archivo="analistajr_competencia_2025.csv.gz"
*  Si usted eligió modalidad *Analista Senior*,  ya se las ingenierá SIN preguntar !

In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets


webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/labo1r"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"


archivo="dataset_pequeno.csv"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

#-------

archivo="gerencial_competencia_2025.csv.gz"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi


## 7.3  Workflow

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [2]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Thu Jul 03 00:36:56 2025"

In [1]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,649910,34.8,1439408,76.9,1070121,57.2
Vcells,1209785,9.3,8388608,64.0,1924953,14.7


In [2]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

Loading required package: data.table

Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.


Attaching package: ‘R.utils’


The following object is masked from ‘package:utils’:

    timestamp


The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, isOpen, nullfile, parse, use, warnings




In [3]:
if (!requireNamespace("readxl", quietly = TRUE)) {
  install.packages("readxl")
}

#### Parametros
Si es gerente, no cambie nada
<br>Si es Analista, cambie el nombre del dataset

In [4]:
PARAM <- list()
PARAM$semilla_primigenia <- 100103

PARAM$experimento <- 7301
PARAM$dataset <- "dataset_analistajr.csv.gz"
PARAM$coef <- "IPC.xlsx"

#### Carpeta del Experimento

In [5]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("WF2_AM", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

### 7.3.1   Preprocesamiento del dataset

#### 7.3.1.1  DT incorporar dataset

In [7]:
# lectura del dataset
library("readxl")

dataset <- fread(paste0("/content/datasets/", PARAM$dataset))
indices <- as.data.table(read_excel(paste0("/content/datasets/", PARAM$coef), sheet = 2))


In [8]:
head(indices)

Mes,coef_IPC,coef_dls_BNA,coef_dls_blue
<dbl>,<dbl>,<dbl>,<dbl>
201901,2.786962,1.374757,5.456790
201902,2.684935,1.369903,5.456790
201903,2.564407,1.395146,5.172840
201904,2.480085,1.424272,4.827160
201905,2.405514,1.444660,4.777778
201906,2.342272,1.445631,4.456790


#### 7.3.1.2  CA  Catastrophe Analysis
Se intentan reparar las variables que para un mes están con todos los valores en cero.

El método que se utiliza es **Machine Learning** se asigna NA also valores, si ha leido bien, es la "anti imputación de valores faltantes"
<br> Usted podrá aplicar aquí otros métodos

In [9]:
dataset[ foto_mes==202006, internet:=NA]
dataset[ foto_mes==202006, mrentabilidad:=NA]
dataset[ foto_mes==202006, mrentabilidad_annual:=NA]
dataset[ foto_mes==202006, mcomisiones:=NA]
dataset[ foto_mes==202006, mactivos_margen:=NA]
dataset[ foto_mes==202006, mpasivos_margen:=NA]
dataset[ foto_mes==202006, mcuentas_saldo:=NA]
dataset[ foto_mes==202006, ctarjeta_visa_transacciones:=NA]
dataset[ foto_mes==202006, mtarjeta_visa_consumo:=NA]
dataset[ foto_mes==202006, mtarjeta_master_consumo:=NA]
dataset[ foto_mes==202006, ccallcenter_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]

#### 7.3.1.3  DR  Data Drifting
Se intenta corregir el data drifting, quizas ajustando por IPC ...
<br>Esta parte podrá ser abordada por todos los Analistas y también la Gerenciapero se decide pedagogicamente no incluirla en esta primer version para reducir la carga cognitiva

In [10]:
# sin codigo en esta primera version del workflow

coef_cols <- setdiff(names(indices), "Mes")

# Detectamos columnas monetarias a ajustar
cols_monetarias <- grep("^(m|Visa_m|Master_m)", names(dataset), value = TRUE)


dataset_versiones <- list()  # lista para guardar versiones

for (coef in coef_cols) {
  
  # Creamos tabla con el coeficiente actual
  indices_reducido <- indices[, .(foto_mes = Mes, ipc_factor = get(coef))]
  setkey(indices_reducido, foto_mes)
  
  # Copiamos el dataset original
  dt <- copy(dataset)
  
  # Traemos el coeficiente correspondiente
  dt[, ipc_factor := indices_reducido[.SD, on = .(foto_mes), ipc_factor]]
  
  # Sobrescribimos las columnas monetarias
  cols_monetarias <- grep("^(m|Visa_m|Master_m)", names(dt), value = TRUE)
  dt[, (cols_monetarias) := lapply(.SD, function(x) x * ipc_factor), .SDcols = cols_monetarias]
  
  # Limpiamos columna temporal
  dt[, ipc_factor := NULL]
  
  # Guardamos esta versión en la lista
  nombre_version <- tolower(gsub("coef_", "", coef))
  dataset_versiones[[nombre_version]] <- dt
}

In [11]:
ncol(dataset_versiones[["ipc"]])

[1] 55

In [15]:
head(dataset_versiones[["ipc"]])
head(dataset)

numero_de_cliente,foto_mes,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,⋯,Visa_mfinanciacion_limite,Visa_Fvencimiento,Visa_msaldototal,Visa_mlimitecompra,Visa_fultimo_cierre,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_mpagominimo,clase_ternaria
<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<chr>
29187730,201901,1,62,296,39857.52,122619.12,25001.336,-3517.7594,16182.886,⋯,1150726.9,-1337,354351.53,1278612.4,22,5604,340170.83,50,31187.277,CONTINUA
29187961,201901,1,59,296,21021.36,95541.58,5465.093,-128.8134,13390.768,⋯,1149330.4,-1915,133011.98,1277060.7,1,7604,135897.04,41,5819.010,CONTINUA
29193101,201901,1,66,349,16024.56,82670.33,13225.445,-10013.4714,12399.752,⋯,577700.2,-1763,174299.79,641889.1,8,1540,74060.87,22,16280.151,CONTINUA
29193281,201901,1,52,142,35513.67,121553.27,34210.100,-2001.5683,3112.869,⋯,105919.1,-1154,48051.16,117687.8,1,2864,55207.74,8,2876.814,CONTINUA
29198891,201901,1,53,296,11810.87,64753.11,3278.192,-1897.6704,9171.920,⋯,141225.4,-211,63558.04,156917.1,1,6756,74553.55,4,2974.887,CONTINUA
29200770,201901,1,56,246,34914.70,168520.74,22966.798,784.8364,9400.925,⋯,1160657.7,-1763,24597.95,1289646.8,8,4474,15579.73,5,4838.278,CONTINUA


numero_de_cliente,foto_mes,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,⋯,Visa_mfinanciacion_limite,Visa_Fvencimiento,Visa_msaldototal,Visa_mlimitecompra,Visa_fultimo_cierre,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_mpagominimo,clase_ternaria
<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<chr>
29187730,201901,1,62,296,14301.42,43997.41,8970.82,-1262.22,5806.64,⋯,412896.5,-1337,127146.16,458783.6,22,5604,122057.93,50,11190.42,CONTINUA
29187961,201901,1,59,296,7542.75,34281.62,1960.95,-46.22,4804.79,⋯,412395.4,-1915,47726.51,458226.8,1,7604,48761.71,41,2087.94,CONTINUA
29193101,201901,1,66,349,5749.83,29663.24,4745.47,-3592.97,4449.20,⋯,207286.7,-1763,62541.14,230318.5,8,1540,26574.05,22,5841.54,CONTINUA
29193281,201901,1,52,142,12742.79,43614.97,12275.05,-718.19,1116.94,⋯,38005.2,-1154,17241.41,42228.0,1,2864,19809.29,8,1032.24,CONTINUA
29198891,201901,1,53,296,4237.90,23234.30,1176.26,-680.91,3291.01,⋯,50673.6,-211,22805.49,56304.0,1,6756,26750.83,4,1067.43,CONTINUA
29200770,201901,1,56,246,12527.87,60467.54,8240.80,281.61,3373.18,⋯,416459.8,-1763,8826.08,462742.9,8,4474,5590.22,5,1736.04,CONTINUA


#### 7.3.1.3  FE_intra_manual Feature Engineering intra-mes

Agrego campos nuevos dentro del mismo mes, SIN considerar la historia.

In [12]:
#####
# Copia para cambiar la version a lista de data.table


atributos_presentes <- function(patributos, dt) {
  atributos <- unique(patributos)
  comun <- intersect(atributos, colnames(dt))
  return(length(atributos) == length(comun))
}

# Elegís un dataset de la lista
dt <- dataset_versiones[["ipc"]]  # podés cambiar a "dls_bna" o "dls_blue"

# Cálculo de kmes (mes del año)
if (atributos_presentes(c("foto_mes"), dt)) {
  dt[, kmes := foto_mes %% 100]
}

# Cálculo de variable derivada
if (atributos_presentes(c("mpayroll", "cliente_edad"), dt)) {
  dt[, mpayroll_sobre_edad := mpayroll / cliente_edad]
}

In [17]:
# visualizo las columas del dataset a esta etapa
colnames(dataset_versiones[["ipc"]])
colnames(dataset)

[1] "numero_de_cliente"                  "foto_mes"                          
 [3] "internet"                           "cliente_edad"                      
 [5] "cliente_antiguedad"                 "mrentabilidad"                     
 [7] "mrentabilidad_annual"               "mcomisiones"                       
 [9] "mactivos_margen"                    "mpasivos_margen"                   
[11] "cproductos"                         "mcuenta_corriente"                 
[13] "mcaja_ahorro"                       "cdescubierto_preacordado"          
[15] "mcuentas_saldo"                     "ctarjeta_visa"                     
[17] "ctarjeta_visa_transacciones"        "mtarjeta_visa_consumo"             
[19] "ctarjeta_master"                    "ctarjeta_master_transacciones"     
[21] "mtarjeta_master_consumo"            "cprestamos_personales"             
[23] "mprestamos_personales"              "cpayroll_trx"                      
[25] "mpayroll"                           "mttarjeta_visa_debitos_automaticos"
[27] "ccomisiones_mantenimiento"          "mcomisiones_mantenimiento"         
[29] "ccomisiones_otras"                  "mtransferencias_recibidas"         
[31] "ccallcenter_transacciones"          "thomebanking"                      
[33] "chomebanking_transacciones"         "ctrx_quarter"                      
[35] "Master_status"                      "Master_mfinanciacion_limite"       
[37] "Master_Fvencimiento"                "Master_msaldototal"                
[39] "Master_mlimitecompra"               "Master_fultimo_cierre"             
[41] "Master_fechaalta"                   "Master_mconsumototal"              
[43] "Master_cconsumos"                   "Master_mpagominimo"                
[45] "Visa_status"                        "Visa_mfinanciacion_limite"         
[47] "Visa_Fvencimiento"                  "Visa_msaldototal"                  
[49] "Visa_mlimitecompra"                 "Visa_fultimo_cierre"               
[51] "Visa_fechaalta"                     "Visa_mconsumototal"                
[53] "Visa_cconsumos"                     "Visa_mpagominimo"                  
[55] "clase_ternaria"                     "kmes"                              
[57] "mpayroll_sobre_edad"

[1] "numero_de_cliente"                  "foto_mes"                          
 [3] "internet"                           "cliente_edad"                      
 [5] "cliente_antiguedad"                 "mrentabilidad"                     
 [7] "mrentabilidad_annual"               "mcomisiones"                       
 [9] "mactivos_margen"                    "mpasivos_margen"                   
[11] "cproductos"                         "mcuenta_corriente"                 
[13] "mcaja_ahorro"                       "cdescubierto_preacordado"          
[15] "mcuentas_saldo"                     "ctarjeta_visa"                     
[17] "ctarjeta_visa_transacciones"        "mtarjeta_visa_consumo"             
[19] "ctarjeta_master"                    "ctarjeta_master_transacciones"     
[21] "mtarjeta_master_consumo"            "cprestamos_personales"             
[23] "mprestamos_personales"              "cpayroll_trx"                      
[25] "mpayroll"                           "mttarjeta_visa_debitos_automaticos"
[27] "ccomisiones_mantenimiento"          "mcomisiones_mantenimiento"         
[29] "ccomisiones_otras"                  "mtransferencias_recibidas"         
[31] "ccallcenter_transacciones"          "thomebanking"                      
[33] "chomebanking_transacciones"         "ctrx_quarter"                      
[35] "Master_status"                      "Master_mfinanciacion_limite"       
[37] "Master_Fvencimiento"                "Master_msaldototal"                
[39] "Master_mlimitecompra"               "Master_fultimo_cierre"             
[41] "Master_fechaalta"                   "Master_mconsumototal"              
[43] "Master_cconsumos"                   "Master_mpagominimo"                
[45] "Visa_status"                        "Visa_mfinanciacion_limite"         
[47] "Visa_Fvencimiento"                  "Visa_msaldototal"                  
[49] "Visa_mlimitecompra"                 "Visa_fultimo_cierre"               
[51] "Visa_fechaalta"                     "Visa_mconsumototal"                
[53] "Visa_cconsumos"                     "Visa_mpagominimo"                  
[55] "clase_ternaria"

In [13]:
#### 

for (nombre in names(dataset_versiones)) {
  dt <- dataset_versiones[[nombre]]
  
  if (atributos_presentes(c("foto_mes"), dt)) {
    dt[, kmes := foto_mes %% 100]
  }
  
  if (atributos_presentes(c("mpayroll", "cliente_edad"), dt)) {
    dt[, mpayroll_sobre_edad := mpayroll / cliente_edad]
  }
  
  # Guardar cambios en la lista
  dataset_versiones[[nombre]] <- dt
}

In [14]:
colnames(dataset_versiones[["ipc"]])

[1] "numero_de_cliente"                  "foto_mes"                          
 [3] "internet"                           "cliente_edad"                      
 [5] "cliente_antiguedad"                 "mrentabilidad"                     
 [7] "mrentabilidad_annual"               "mcomisiones"                       
 [9] "mactivos_margen"                    "mpasivos_margen"                   
[11] "cproductos"                         "mcuenta_corriente"                 
[13] "mcaja_ahorro"                       "cdescubierto_preacordado"          
[15] "mcuentas_saldo"                     "ctarjeta_visa"                     
[17] "ctarjeta_visa_transacciones"        "mtarjeta_visa_consumo"             
[19] "ctarjeta_master"                    "ctarjeta_master_transacciones"     
[21] "mtarjeta_master_consumo"            "cprestamos_personales"             
[23] "mprestamos_personales"              "cpayroll_trx"                      
[25] "mpayroll"                           "mttarjeta_visa_debitos_automaticos"
[27] "ccomisiones_mantenimiento"          "mcomisiones_mantenimiento"         
[29] "ccomisiones_otras"                  "mtransferencias_recibidas"         
[31] "ccallcenter_transacciones"          "thomebanking"                      
[33] "chomebanking_transacciones"         "ctrx_quarter"                      
[35] "Master_status"                      "Master_mfinanciacion_limite"       
[37] "Master_Fvencimiento"                "Master_msaldototal"                
[39] "Master_mlimitecompra"               "Master_fultimo_cierre"             
[41] "Master_fechaalta"                   "Master_mconsumototal"              
[43] "Master_cconsumos"                   "Master_mpagominimo"                
[45] "Visa_status"                        "Visa_mfinanciacion_limite"         
[47] "Visa_Fvencimiento"                  "Visa_msaldototal"                  
[49] "Visa_mlimitecompra"                 "Visa_fultimo_cierre"               
[51] "Visa_fechaalta"                     "Visa_mconsumototal"                
[53] "Visa_cconsumos"                     "Visa_mpagominimo"                  
[55] "clase_ternaria"                     "kmes"                              
[57] "mpayroll_sobre_edad"

#### 7.3.1.4  FE_rf Feature Engineering de nuevas variables a partir de hojas de Random Forest

Esto se mostrará unicamente a la *modalidad Analista Sr*

In [ ]:
# No se implementa Feature Engineering a partir de Random Forest

#### 7.3.1.5  FEhist Feature Engineering historico

El Fature Engineering Histórico es la etapa que más aporta a la ganancia final, ya que enriquece cada registro del dataset con su historia.

Para cada campo del dataset original (*)
se crean lo siguientes campos de a partir de la historia
* lag1  lags de orden 1
* delta1  =  valor actual - lag1
* lag2  lags de orden 2
* delta2  = valor actual - lag2


(*) Excepto para los campos  <numero_de_cliente,  foto_mes,  clase_ternaria>

In [15]:
#######
# Version para lista de datasets 
#Feature engineering historico

for (nombre in names(dataset_versiones)) {
  dt <- dataset_versiones[[nombre]]
  
  cols_lagueables <- setdiff(
    colnames(dt),
    c("numero_de_cliente", "foto_mes", "clase_ternaria")
  )
  
  dt[, paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
     by = numero_de_cliente,
     .SDcols = cols_lagueables]
  
  dt[, paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
     by = numero_de_cliente,
     .SDcols = cols_lagueables]
  
  for (vcol in cols_lagueables) {
    dt[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dt[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
  }
  
  dataset_versiones[[nombre]] <- dt
}

Verificacion de los campos recien creados

In [16]:
ncol(dataset_versiones[["ipc"]])
colnames(dataset_versiones[["ipc"]])
ncol(dataset)
colnames(dataset)

[1] 273

[1] "numero_de_cliente"                        
  [2] "foto_mes"                                 
  [3] "internet"                                 
  [4] "cliente_edad"                             
  [5] "cliente_antiguedad"                       
  [6] "mrentabilidad"                            
  [7] "mrentabilidad_annual"                     
  [8] "mcomisiones"                              
  [9] "mactivos_margen"                          
 [10] "mpasivos_margen"                          
 [11] "cproductos"                               
 [12] "mcuenta_corriente"                        
 [13] "mcaja_ahorro"                             
 [14] "cdescubierto_preacordado"                 
 [15] "mcuentas_saldo"                           
 [16] "ctarjeta_visa"                            
 [17] "ctarjeta_visa_transacciones"              
 [18] "mtarjeta_visa_consumo"                    
 [19] "ctarjeta_master"                          
 [20] "ctarjeta_master_transacciones"            
 [21] "mtarjeta_master_consumo"                  
 [22] "cprestamos_personales"                    
 [23] "mprestamos_personales"                    
 [24] "cpayroll_trx"                             
 [25] "mpayroll"                                 
 [26] "mttarjeta_visa_debitos_automaticos"       
 [27] "ccomisiones_mantenimiento"                
 [28] "mcomisiones_mantenimiento"                
 [29] "ccomisiones_otras"                        
 [30] "mtransferencias_recibidas"                
 [31] "ccallcenter_transacciones"                
 [32] "thomebanking"                             
 [33] "chomebanking_transacciones"               
 [34] "ctrx_quarter"                             
 [35] "Master_status"                            
 [36] "Master_mfinanciacion_limite"              
 [37] "Master_Fvencimiento"                      
 [38] "Master_msaldototal"                       
 [39] "Master_mlimitecompra"                     
 [40] "Master_fultimo_cierre"                    
 [41] "Master_fechaalta"                         
 [42] "Master_mconsumototal"                     
 [43] "Master_cconsumos"                         
 [44] "Master_mpagominimo"                       
 [45] "Visa_status"                              
 [46] "Visa_mfinanciacion_limite"                
 [47] "Visa_Fvencimiento"                        
 [48] "Visa_msaldototal"                         
 [49] "Visa_mlimitecompra"                       
 [50] "Visa_fultimo_cierre"                      
 [51] "Visa_fechaalta"                           
 [52] "Visa_mconsumototal"                       
 [53] "Visa_cconsumos"                           
 [54] "Visa_mpagominimo"                         
 [55] "clase_ternaria"                           
 [56] "kmes"                                     
 [57] "mpayroll_sobre_edad"                      
 [58] "internet_lag1"                            
 [59] "cliente_edad_lag1"                        
 [60] "cliente_antiguedad_lag1"                  
 [61] "mrentabilidad_lag1"                       
 [62] "mrentabilidad_annual_lag1"                
 [63] "mcomisiones_lag1"                         
 [64] "mactivos_margen_lag1"                     
 [65] "mpasivos_margen_lag1"                     
 [66] "cproductos_lag1"                          
 [67] "mcuenta_corriente_lag1"                   
 [68] "mcaja_ahorro_lag1"                        
 [69] "cdescubierto_preacordado_lag1"            
 [70] "mcuentas_saldo_lag1"                      
 [71] "ctarjeta_visa_lag1"                       
 [72] "ctarjeta_visa_transacciones_lag1"         
 [73] "mtarjeta_visa_consumo_lag1"               
 [74] "ctarjeta_master_lag1"                     
 [75] "ctarjeta_master_transacciones_lag1"       
 [76] "mtarjeta_master_consumo_lag1"             
 [77] "cprestamos_personales_lag1"               
 [78] "mprestamos_personales_lag1"               
 [79] "cpayroll_trx_lag1"                        
 [80] "mpayroll_lag1"                            
 [

[1] 55

[1] "numero_de_cliente"                  "foto_mes"                          
 [3] "internet"                           "cliente_edad"                      
 [5] "cliente_antiguedad"                 "mrentabilidad"                     
 [7] "mrentabilidad_annual"               "mcomisiones"                       
 [9] "mactivos_margen"                    "mpasivos_margen"                   
[11] "cproductos"                         "mcuenta_corriente"                 
[13] "mcaja_ahorro"                       "cdescubierto_preacordado"          
[15] "mcuentas_saldo"                     "ctarjeta_visa"                     
[17] "ctarjeta_visa_transacciones"        "mtarjeta_visa_consumo"             
[19] "ctarjeta_master"                    "ctarjeta_master_transacciones"     
[21] "mtarjeta_master_consumo"            "cprestamos_personales"             
[23] "mprestamos_personales"              "cpayroll_trx"                      
[25] "mpayroll"                           "mttarjeta_visa_debitos_automaticos"
[27] "ccomisiones_mantenimiento"          "mcomisiones_mantenimiento"         
[29] "ccomisiones_otras"                  "mtransferencias_recibidas"         
[31] "ccallcenter_transacciones"          "thomebanking"                      
[33] "chomebanking_transacciones"         "ctrx_quarter"                      
[35] "Master_status"                      "Master_mfinanciacion_limite"       
[37] "Master_Fvencimiento"                "Master_msaldototal"                
[39] "Master_mlimitecompra"               "Master_fultimo_cierre"             
[41] "Master_fechaalta"                   "Master_mconsumototal"              
[43] "Master_cconsumos"                   "Master_mpagominimo"                
[45] "Visa_status"                        "Visa_mfinanciacion_limite"         
[47] "Visa_Fvencimiento"                  "Visa_msaldototal"                  
[49] "Visa_mlimitecompra"                 "Visa_fultimo_cierre"               
[51] "Visa_fechaalta"                     "Visa_mconsumototal"                
[53] "Visa_cconsumos"                     "Visa_mpagominimo"                  
[55] "clase_ternaria"

#### 7.3.1.6  FEhist Reduccion dimensionalidad con canaritos

Esta etapa solo se mostrará a la *modalidad Anlista Sr* por algun canal secreto de forma de no confundir a los *Analista Jr*  nni distraer con detalles operativos a la estratégica *Modalidad Gerencial*

In [ ]:
# No se implementa la reduccion de la dimensionalidad con canaritos

### 7.3.2 Modelado

#### 7.3.2.1 Training Strategy

Esta etapa de Workflow de  Training Strategy esta pensada para la *Modalidad Gerencial* que posee el dataset de [202005, 202109]
<br> Si usted es un Analista, posee el periodo de [201901, 202109] y deberá experimentar en que meses le conviene experimentar

<br> A la *Modalidad Gerencial* no se le complicada la vida con el undersampling de los continua, por eso PARAM$trainingstrategy$training_pct <- 1.0
<br> Sin embargo, si usted es  *Analista SR* posee un dataset 50 veces ( filas x columnas) más grande que la *Modalidad Gerencial*  y por un tema de velocidad y experimentación más rápida puede llegar a necesitar activar el undersampling de la clase mayoritaria, a pesar de estar corriendo en Google Cloud.

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202109  obviamente completo

* final_train =  [ 202005, 202107 ]  con un undersampling de los CONTINUA del 2% por un tema de velocidad

* training
   * testing = NO HAY
   * validation =  202107   completo, sin undersampling
   * training = [ 202005, 202106 ]  donde se consideran el 100% de los CONTINUA

In [16]:
PARAM$trainingstrategy$validate <- c(202109)

PARAM$trainingstrategy$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906, 
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106,
  202107
)

PARAM$trainingstrategy$training_pct <- 1


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [17]:
####
# Clase ternaria en los nuevos data.table. 

for (nombre in names(dataset_versiones)) {
  dt <- dataset_versiones[[nombre]]
  
  dt[, clase01 := ifelse(clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0)]
  
  # Guardamos la modificación de vuelta en la lista
  dataset_versiones[[nombre]] <- dt
}


In [22]:
table(dataset_versiones[["ipc"]]$clase01)
table(dataset$clase01)


     0      1 
967988   8386 

< table of extent 0 >

In [18]:
# Campos buenos para la lista de data.table

# Campos que no queremos usar como predictores
excluir <- c("clase_ternaria", "clase01", "azar")

# Creamos una lista vacía para guardar los campos buenos por versión
campos_buenos_versiones <- list()

# Iteramos sobre cada dataset dentro de la lista
for (nombre in names(dataset_versiones)) {
  dt <- dataset_versiones[[nombre]]
  
  campos_buenos <- setdiff(colnames(dt), excluir)
  campos_buenos_versiones[[nombre]] <- copy(campos_buenos)
}

In [19]:
##############
# Adaptacion bloque anterior para lista de data.table

if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

# Lista donde vas a guardar los dtrain para cada versión
dtrain_versiones <- list()

# Preparación por cada dataset
for (nombre in names(dataset_versiones)) {
  
  dt <- copy(dataset_versiones[[nombre]])  # trabajamos sobre una copia
  
  # Paso 1: azar (undersampling)
  set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
  dt[, azar := runif(.N)]
  
  # Paso 2: fold de entrenamiento
  dt[, fold_train := foto_mes %in% PARAM$trainingstrategy$training &
                        (clase_ternaria %in% PARAM$trainingstrategy$positivos |
                         azar < PARAM$trainingstrategy$training_pct)]
  
  # Paso 3: dataset de entrenamiento para LightGBM
  campos <- campos_buenos_versiones[[nombre]]
  
  dtrain <- lgb.Dataset(
    data = data.matrix(dt[fold_train == TRUE, ..campos]),
    label = dt[fold_train == TRUE, clase01],
    free_raw_data = TRUE
  )
  
  # Guardamos el dtrain en la lista
  dtrain_versiones[[nombre]] <- dtrain
  
  # (Opcional) Guardamos de vuelta el dataset con azar/fold en la lista
  dataset_versiones[[nombre]] <- dt
}

Loading required package: lightgbm



In [20]:
####### 
#Datos de validacion 
#Adaptacion a la lista de data.table

# Creamos una lista vacía para guardar los dvalidate
dvalidate_versiones <- list()

for (nombre in names(dataset_versiones)) {
  
  dt <- dataset_versiones[[nombre]]
  campos <- campos_buenos_versiones[[nombre]]
  
  dvalidate <- lgb.Dataset(
    data = data.matrix(dt[foto_mes %in% PARAM$trainingstrategy$validate, ..campos]),
    label = dt[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
    free_raw_data = TRUE
  )
  
  dvalidate_versiones[[nombre]] <- dvalidate
}

In [21]:
nrow(dvalidate_versiones[["ipc"]])

[1] 33094

####  7.3.2.2. Hyperparameter Tuning

* Clase binaria que se optimiza :  positivos = [ BAJA+1, BAJA+2 ]

* Metrica que se optimiza **AUC** Area Under Curve de la  ROC Curve

es muy importante notar que intencionalmente  **NO** se está optimizando la funcion de ganancia del problema

* Cantidad de iteraciones inteligentes de la Optimizacion Bayesiana = **10**

* Parametros no default, fijos de LightGBM que no se optimizan
  * max_bin = 31 , Alienigenas Ancestrales contruyeron las pirámides y dejaron a la humanidad en un jeroglifico  *max_bin=31*
  * feature_fraction = 0.5  para poner algo que generalmente no falla
  * learning_rate = 0.03  para que aprenda lento


* Parametros que se optimizan en la Bayesian Optimization
  * num_leaves  [8, 256]
  * min_data_in_leaf  [8, 8192]

In [22]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: DiceKriging

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers


Attaching package: ‘ParamHelpers’


The following object is masked from ‘package:R.utils’:

    isVector



Attaching package: ‘mlr’


The following objects are masked from ‘package:R.utils’:

    resample, setThreshold


Loading required package: smoof

Loading required package: checkmate


Attaching package: ‘checkmate’


The following object is masked from ‘package:DiceKriging’:

    checkNames


The following object is masked from ‘package:R.utils’:

    asInt



Attaching package: ‘smoof’


The following objects are masked from ‘package:R.oo’:

    getDescription, getName




Definición de la Bayesian Optimization
<br> Si se desea optimizar un hiperparámetro que esta como fijo, debe QUITARSE de param_fijos y agregarse a PARAM$hipeparametertuning$hs

In [23]:
# valor ridiculamente bajo para que corra rapido en el aula y no molestar a la *Modalidad Gerencial*
PARAM$hipeparametertuning$num_interations <- 10

# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "auc",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  max_bin= 31,
  learning_rate= 0.03,
  feature_fraction= 0.5,
  num_iterations= 2048,  # valor grande, lo limita early_stopping_rounds
  early_stopping_rounds= 200
)

PARAM$hipeparametertuning$hs <- makeParamSet(
  makeIntegerParam("num_leaves", lower = 2L, upper = 256L),
  makeIntegerParam("min_data_in_leaf", lower = 2L, upper = 8192L)
)

Función "señora caja negra"  que es llamada para verificar la realidad por la Bayesian Optimization

In [24]:
######
# Adaptacion 
EstimarGanancia_AUC_lightgbm <- function(x, nombre_version) {

  message(paste0(format(Sys.time(), "%a %b %d %X %Y"), " - ", nombre_version))

  # Uno hiperparámetros fijos + los que se están optimizando
  param_completo <- c(PARAM$lgbm$param_fijos, x)

  # Entreno LightGBM con los datasets correspondientes a esta versión
  modelo_train <- lgb.train(
    data = dtrain_versiones[[nombre_version]],
    valids = list(valid = dvalidate_versiones[[nombre_version]]),
    eval = "auc",
    param = param_completo,
    verbose = -100
  )

  # Recupero el AUC en validation
  AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

  # Devuelvo el número de iteraciones como atributo adicional
  attr(AUC, "extras") <- list("num_iterations" = modelo_train$best_iter)

  # Limpieza de memoria
  rm(modelo_train)
  gc(full = TRUE, verbose = FALSE)

  return(AUC)
}

seteo de la Bayesian Optimization (complejo)
<br> copiado y pegado de la documentación de la librería

In [25]:
configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
    fn= EstimarGanancia_AUC_lightgbm, # la funcion que voy a maximizar
    minimize= FALSE, # estoy Maximizando AUC
    noisy= FALSE,
    par.set= PARAM$hipeparametertuning$hs,
    has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
    save.on.disk.at.time= 600,
    save.file.path= "HT_AM.RDATA"
)

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
    ctrl,
    iters= PARAM$hipeparametertuning$num_interations  # cantidad de iteraciones inteligentes
)

# defino el método estandar para la creacion de los puntos iniciales
#   los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# mas configuraciones
surr.km <- makeLearner(
    "regr.km",
    predict.type= "se",
    covtype= "matern3_2",
    control= list(trace = TRUE)
)

Corrida de la Bayesian Optimization,  aqui se hace el trabajo pesado
<br> por favor no se asuste con los warnings que pudieran aparecer

Si corrío a medias y llegó a las iteraciones inteligentes, en el archivo binario HT.RDATA quedó lo ya procesado y es utilizado para retomar la corrida desde lo último que llegó a grabar.

In [26]:
#### Funcion para aplicarlo a cada dataset 

realizar_busqueda <- function(nombre_version) {
  
  archivo_ht <- paste0("HT_", nombre_version, ".RDATA")
  
  # Creo la función objetivo específica para esta versión
  obj.fun <- makeSingleObjectiveFunction(
    fn = function(x) EstimarGanancia_AUC_lightgbm(x, nombre_version),
    minimize = FALSE,
    noisy = FALSE,
    par.set = PARAM$hipeparametertuning$hs,
    has.simple.signature = FALSE
  )
  
  # Configuro el control con guardado específico por versión
  ctrl_local <- ctrl
  ctrl_local$save.file.path <- archivo_ht

  if (!file.exists(archivo_ht)) {
    bayesiana_salida <- mbo(obj.fun, learner = surr.km, control = ctrl_local)
  } else {
    bayesiana_salida <- mboContinue(archivo_ht)
  }
  
  return(bayesiana_salida)
}

In [27]:
########
#### Corrida para la lista de data.table

resultados_bayesiana <- list()

for (nombre in names(dataset_versiones)) {
  resultados_bayesiana[[nombre]] <- realizar_busqueda(nombre)
}

Warning message in mboContinue(archivo_ht):
“Tuning ended with term.iter. No need to continue. Simply returning stored result.”
Warning message in mboContinue(archivo_ht):
“Tuning ended with term.iter. No need to continue. Simply returning stored result.”
Warning message in mboContinue(archivo_ht):
“Tuning ended with term.iter. No need to continue. Simply returning stored result.”


In [33]:
################
# Inicializo donde voy a guardar los mejores hiperparámetros
PARAM$out$lgbm$mejores_hiperparametros <- list()

for (nombre in names(resultados_bayesiana)) {
  
  bayesiana_salida <- resultados_bayesiana[[nombre]]
  
  # Extraigo el historial como data.table
  tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
  
  # Ordeno por AUC y luego por iteraciones (más por estabilidad que necesidad)
  setorder(tb_bayesiana, -y, -num_iterations)
  
  # Guardo los logs por separado
  fwrite(tb_bayesiana,
         file = paste0("BO_log_AM", nombre, ".txt"),
         sep = "\t")
  
  # Extraigo los mejores hiperparámetros (pueden incluir más columnas si las tenés)
  PARAM$out$lgbm$mejores_hiperparametros[[nombre]] <- tb_bayesiana[1,
    .(num_leaves, min_data_in_leaf, num_iterations)]
  
  print(paste("Mejores hiperparámetros para", nombre))
  print(PARAM$out$lgbm$mejores_hiperparametros[[nombre]])
}

[1] "Mejores hiperparámetros para ipc"
   num_leaves min_data_in_leaf num_iterations
        <int>            <int>          <int>
1:        120             6500              1
[1] "Mejores hiperparámetros para dls_bna"
   num_leaves min_data_in_leaf num_iterations
        <int>            <int>          <int>
1:        118             4835              1
[1] "Mejores hiperparámetros para dls_blue"
   num_leaves min_data_in_leaf num_iterations
        <int>            <int>          <int>
1:         87             7171              1


la bayesian optimization ha corrido, extraigo los mejores hiperparametros

### 7.3.3 Produccion

### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

#### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

In [34]:
##############Adaptacion para lista de data.frame 

# Definís los meses del entrenamiento final (esto no cambia)
PARAM$trainingstrategy$final_train <- c( 
  201901, 201902, 201903, 201904, 201905, 201906, 
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106,
  202107
)

# Lista para guardar los datasets de entrenamiento final
dfinal_train_versiones <- list()

# Recorro cada versión del dataset
for (nombre in names(dataset_versiones)) {
  dt <- dataset_versiones[[nombre]]
  campos <- campos_buenos_versiones[[nombre]]
  
  # Genero la variable fold_final_train
  dt[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train]
  
  # Creo el dataset final para lightGBM
  dfinal_train <- lgb.Dataset(
    data = data.matrix(dt[fold_final_train == TRUE, ..campos]),
    label = dt[fold_final_train == TRUE, clase01],
    free_raw_data = TRUE
  )
  
  # Guardo en la lista
  dfinal_train_versiones[[nombre]] <- dfinal_train
  
  # Guardo de vuelta el dataset modificado
  dataset_versiones[[nombre]] <- dt
}

In [38]:
# ver cuántas observaciones tiene el final train de cada versión

sapply(dfinal_train_versiones, class)
sapply(dfinal_train_versiones, function(d) length(d$get_field("label")))


ipc,dls_bna,dls_blue
lgb.Dataset,lgb.Dataset,lgb.Dataset
R6,R6,R6


ipc  dls_bna dls_blue 
  910211   910211   910211

#### Final Training Hyperparameters

In [35]:
########Adaptacion

param_final_versiones <- list()

for (nombre in names(dataset_versiones)) {
  fijos <- copy(PARAM$lgbm$param_fijos)
  
  # Quito lo que fue optimizado
  fijos$num_iterations <- NULL
  fijos$early_stopping_rounds <- NULL
  
  # Combino con los mejores hiperparámetros encontrados para esta versión
  mejores <- PARAM$out[[nombre]]$lgbm$mejores_hiperparametros
  
  param_final_versiones[[nombre]] <- c(fijos, mejores)
}

#### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

In [36]:
########Adaptacion
final_models_versiones <- list()

for (nombre in names(dfinal_train_versiones)) {
  cat("Entrenando modelo final para:", nombre, "\n")
  
  final_models_versiones[[nombre]] <- lgb.train(
    data = dfinal_train_versiones[[nombre]],
    param = param_final_versiones[[nombre]],
    verbose = -100
  )
}

Entrenando modelo final para: ipc 
Entrenando modelo final para: dls_bna 
Entrenando modelo final para: dls_blue 


In [37]:
####Guardar los modelos entrenados en archivos 

for (nombre in names(final_models_versiones)) {
  modelo <- final_models_versiones[[nombre]]
  lgb.save(modelo, paste0("modelo_final_", nombre, ".txt"))
}

In [ ]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...

lgb.save(final_model, "modelo_AM.txt")

In [38]:
importancias_versiones <- list()  # Lista para guardar todas las tablas

for (nombre in names(final_models_versiones)) {
  
  modelo <- final_models_versiones[[nombre]]
  
  # Calcular importancia de variables
  tb_importancia <- as.data.table(lgb.importance(modelo))
  
  # Guardar en la lista
  importancias_versiones[[nombre]] <- tb_importancia
  
  # (Opcional) Guardar en archivo
  archivo_importancia <- paste0("impo_", nombre, ".txt")
  fwrite(tb_importancia,
         file = archivo_importancia,
         sep = "\t")
}


In [39]:
importancias_versiones[["ipc"]]

Feature,Gain,Cover,Frequency
<chr>,<dbl>,<dbl>,<dbl>
ctrx_quarter,0.225727525,1.106582e-01,0.021000000
mcaja_ahorro,0.080012607,8.352161e-02,0.026333333
mtarjeta_visa_consumo,0.076015900,5.754273e-02,0.022333333
mcaja_ahorro_lag1,0.051465589,9.728501e-03,0.005000000
mcuentas_saldo,0.046483811,2.735251e-02,0.019666667
Visa_mfinanciacion_limite,0.039684292,2.815727e-03,0.005000000
cpayroll_trx,0.032298197,7.243183e-02,0.021333333
foto_mes,0.025449378,2.912310e-02,0.056000000
mcuenta_corriente,0.019749147,1.190779e-02,0.022666667


### Scoring

Aplico el modelo final a los datos del futuro

In [38]:
#####
#Prediccion a varios modelos
PARAM$trainingstrategy$future <- c(202109)

# Lista donde guardar las predicciones por versión
predicciones_versiones <- list()

for (nombre in names(dataset_versiones)) {
  dt <- dataset_versiones[[nombre]]
  campos <- campos_buenos_versiones[[nombre]]
  modelo <- final_models_versiones[[nombre]]
  
  # Filtro los datos del futuro
  dfuture <- dt[foto_mes %in% PARAM$trainingstrategy$future]
  
  # Aplico el modelo
  predicciones_versiones[[nombre]] <- predict(
    modelo,
    data.matrix(dfuture[, ..campos])
  )
}

In [39]:
###### 
for (nombre in names(dataset_versiones)) {
  dt <- dataset_versiones[[nombre]]
  campos <- campos_buenos_versiones[[nombre]]
  modelo <- final_models_versiones[[nombre]]
  
  dt[foto_mes %in% PARAM$trainingstrategy$future,
     pred := predict(modelo, data.matrix(.SD)),
     .SDcols = campos]
  
  dataset_versiones[[nombre]] <- dt
}

#### Tabla Prediccion

In [40]:
#####

for (nombre in names(dataset_versiones)) {
  dt <- dataset_versiones[[nombre]]
  pred <- predicciones_versiones[[nombre]]
  
  # Filtro solo los del mes futuro
  dfuture <- dt[foto_mes %in% PARAM$trainingstrategy$future]
  
  # Armo la tabla de predicción
  tb_prediccion <- dfuture[, .(numero_de_cliente)]
  tb_prediccion[, prob := pred]
  
  # Guardo en archivo por versión
  fwrite(
    tb_prediccion,
    file = paste0("prediccion_", nombre, ".txt"),
    sep = "\t"
  )
}

Kaggle Competition Submit

Genero las salidas y hago los submits a Kaggle
<br>El notebook esta preparado para la Modalidad Gerencial, los analistas deben hacer cambios.
<br> Los analistas deben cambiar **competencia** a SU competencia  "labo-i-2025-rosario-analista-jr"   o  la original "labo-i-2025-rosario"  para los Senior
<br> Los cortes  dependen de la cantidad de registros, multiplicar por 2 para los Analistas Jr y por 10 para los Analista Sr

Los Analista Sr luego de meditar cuidadosamente reducirán la cantidad de cortes

In [41]:
#######
PARAM$kaggle$competencia <- "labo-i-2025-rosario-analista-jr"
PARAM$kaggle$cortes <- 2 * seq(800, 1300, by = 50)

dir.create("kaggle", showWarnings = FALSE)

for (nombre in names(predicciones_versiones)) {
  dt <- dataset_versiones[[nombre]]
  pred <- predicciones_versiones[[nombre]]
  
  dfuture <- dt[foto_mes %in% PARAM$trainingstrategy$future]
  tb_prediccion <- dfuture[, .(numero_de_cliente)]
  tb_prediccion[, prob := pred]
  
  # ordeno por probabilidad descendente
  setorder(tb_prediccion, -prob)

  for (envios in PARAM$kaggle$cortes) {
    tb_prediccion[, Predicted := 0L]
    tb_prediccion[1:envios, Predicted := 1L]

    archivo_kaggle <- sprintf("./kaggle/KA_%s_%s_%s.csv", 
                              PARAM$experimento, nombre, envios)

    fwrite(tb_prediccion[, .(numero_de_cliente, Predicted)],
           file = archivo_kaggle, sep = ",")

    # subir a Kaggle (opcional, requiere CLI autenticada)
    comando <- sprintf("kaggle competitions submit -c %s -f %s -m 'envios=%s semilla=%s version=%s'",
                       PARAM$kaggle$competencia,
                       archivo_kaggle,
                       envios,
                       PARAM$semilla_primigenia,
                       nombre)

    salida <- system(comando, intern = TRUE)
    cat(salida, "\n")
  }
}

Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 


Warning message in system(comando, intern = TRUE):
“running command 'kaggle competitions submit -c labo-i-2025-rosario-analista-jr -f ./kaggle/KA_7301_dls_bna_2100.csv -m 'envios=2100 semilla=100103 version=dls_bna'' had status 1”


500 Server Error: Internal Server Error for url: https://www.kaggle.com/api/v1/competitions/submissions/submit/labo-i-2025-rosario-analista-jr 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario Analista JR 
Successfully submitted to LaboI 2025 Rosario

In [164]:
# grabo los parametros
if( !require("yaml")) install.packages("yaml")
require("yaml")

write_yaml( PARAM, file="PARAM.yml")

In [55]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Fri Jun 06 20:29:27 2025"